<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Insects Results

_by Jeremy Freeman (CarbonPlan), October 26, 2020_

This notebook loads insects results.


In [2]:
import numpy as np
import pandas as pd
from carbonplan_forests import setup, plot, load, fit, utils, prepare

In [9]:
setup.plotting(remote=False)

In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Load the raw FIA grouped data


In [38]:
pf = pd.read_parquet(
    "/Users/freeman/github/carbonplan/forests/scripts/data/insects.parquet"
)

Filter out bad values

In [39]:
badinds = (pf['historical'] > 1) | (np.isnan(pf['historical']))
for key in pf.columns:
    if key not in ['lat', 'lon', 'type_code', 'r2']:
        badinds = badinds | ((pf[key] > 1) | (np.isnan(pf[key])))
pf = pf[~badinds]

In [48]:
pf.columns

Index(['lat', 'lon', 'type_code', 'historical', 'BCC-CSM2-MR_ssp245_2020',
       'BCC-CSM2-MR_ssp370_2020', 'BCC-CSM2-MR_ssp585_2020',
       'BCC-CSM2-MR_ssp245_2040', 'BCC-CSM2-MR_ssp370_2040',
       'BCC-CSM2-MR_ssp585_2040', 'BCC-CSM2-MR_ssp245_2060',
       'BCC-CSM2-MR_ssp370_2060', 'BCC-CSM2-MR_ssp585_2060',
       'BCC-CSM2-MR_ssp245_2080', 'BCC-CSM2-MR_ssp370_2080',
       'BCC-CSM2-MR_ssp585_2080', 'BCC-CSM2-MR_ssp245_2100',
       'BCC-CSM2-MR_ssp370_2100', 'BCC-CSM2-MR_ssp585_2100', 'r2'],
      dtype='object')

Make a map of the historical prediction (from the period used for fitting)

In [60]:
plot.carto(
    lat=pf["lat"],
    lon=pf["lon"],
    color=pf["historical"],
    cmap="reds",
    clim=(0,0.3),
    size=2,
)

alt.LayerChart(...)

Make a time series of spatially averaged projections

In [64]:
years = np.arange(2020,2120,20)
y1 = [pf['BCC-CSM2-MR_ssp245_' + str(year)].mean() for year in years]
y2 = [pf['BCC-CSM2-MR_ssp370_' + str(year)].mean() for year in years]
y3 = [pf['BCC-CSM2-MR_ssp585_' + str(year)].mean() for year in years]
opts = {'color': 'rgb(164,69,70)', 'ylabel': 'fraction insects', 'xlabel': 'year'}
(
plot.line(x=years, y=y1, opacity=0.5, ylim=[0.02,0.04], **opts)
+ plot.line(x=years, y=y2, opacity=0.75, **opts)
+ plot.line(x=years, y=y3, opacity=1, **opts)
)

alt.LayerChart(...)